In [2]:
!pip install pydub numpy scipy
!pip install librosa numpy


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install flask fastapi uvicorn requests pybluez pymongo


  Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
  Using cached PyBluez-0.23.tar.gz (97 kB)
  Running setup.py clean for pybluez
Failed to build pybluez
    Running setup.py install for pybluez: started
    Running setup.py install for pybluez: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-e4faze4m\\pybluez_2f4620e52bd7415c9fd684d00bf1749a\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-e4faze4m\\pybluez_2f4620e52bd7415c9fd684d00bf1749a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\User\AppData\Local\Temp\pip-wheel-v6em763m'
       cwd: C:\Users\User\AppData\Local\Temp\pip-install-e4faze4m\pybluez_2f4620e52bd7415c9fd684d00bf1749a\
  Complete output (28 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating buil

In [4]:
!pip install pyaudio


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from flask import Flask, request, jsonify
import os
import cv2  # For capturing images via camera
import pyaudio  # For capturing audio
import wave  # For audio file saving
import librosa  # For audio processing
import numpy as np  # For numerical operations

app = Flask(__name__)

# Directory where images and audio will be saved
image_upload_folder = 'uploads/images'
audio_upload_folder = 'uploads/audio'

# Create directories if they don't exist
os.makedirs(image_upload_folder, exist_ok=True)
os.makedirs(audio_upload_folder, exist_ok=True)

@app.route("/")
def home():
    return "Raspberry Pi Web Server is Running!"

# Endpoint to receive sensor data
@app.route("/sensor_data", methods=["POST"])
def receive_data():
    data = request.json
    print("Received Data:", data)
    return jsonify({"status": "success", "message": "Data received!"})

# Endpoint for uploading audio file
@app.route('/upload_audio', methods=['POST'])
def upload_audio():
    if 'audio' not in request.files:
        return "No audio file found", 400
    audio = request.files['audio']
    if audio.filename == '':
        return "No selected file", 400
    audio_path = os.path.join(audio_upload_folder, audio.filename)
    audio.save(audio_path)

    # Perform audio processing (fire alarm detection)
    alarm_detected = process_audio(audio_path)
    if alarm_detected:
        return jsonify({"status": "success", "message": "Audio file successfully uploaded, Fire alarm detected!"}), 200
    else:
        return jsonify({"status": "success", "message": "Audio file successfully uploaded, No fire alarm detected."}), 200

# Endpoint for uploading image file
@app.route('/upload_image', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return "No image file found", 400
    image = request.files['image']
    if image.filename == '':
        return "No selected file", 400
    image_path = os.path.join(image_upload_folder, image.filename)
    image.save(image_path)
    return "Image file successfully uploaded", 200

# Endpoint to capture audio automatically (using pyaudio)
@app.route('/capture_audio', methods=['GET'])
def capture_audio():
    # Set up audio recording (e.g., 5 seconds of recording)
    duration = 5  # seconds
    fs = 44100  # Sample rate (standard for audio recording)

    # Set up PyAudio for recording
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=fs,
                    input=True,
                    frames_per_buffer=1024)
    
    print("Recording...")
    frames = []
    for i in range(0, int(fs / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    print("Recording complete.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the captured audio to a .wav file
    audio_filename = "captured_audio.wav"
    audio_path = os.path.join(audio_upload_folder, audio_filename)
    wf = wave.open(audio_path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

    # Perform audio processing (fire alarm detection)
    alarm_detected = process_audio(audio_path)
    if alarm_detected:
        return jsonify({"status": "success", "message": "Audio captured and saved, Fire alarm detected.", "file_path": audio_path}), 200
    else:
        return jsonify({"status": "success", "message": "Audio captured and saved, No fire alarm detected.", "file_path": audio_path}), 200

# Endpoint to capture image automatically (using OpenCV)
@app.route('/capture_image', methods=['GET'])
def capture_image():
    # Initialize the camera (0 is the default camera)
    cap = cv2.VideoCapture(0)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        return jsonify({"status": "error", "message": "Could not access camera."}), 400

    # Capture a frame
    ret, frame = cap.read()
    if not ret:
        return jsonify({"status": "error", "message": "Failed to capture image."}), 400

    # Save the captured image
    image_filename = "captured_image.jpg"
    image_path = os.path.join(image_upload_folder, image_filename)
    cv2.imwrite(image_path, frame)

    # Release the camera and return a response
    cap.release()
    return jsonify({"status": "success", "message": "Image captured and saved.", "file_path": image_path}), 200

def process_audio(audio_path):
    # Load the audio file for processing
    audio, sr = librosa.load(audio_path, sr=None)

    # Compute the Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(audio))

    # Compute the Mel-frequency cepstral coefficients (MFCCs) for detecting features like fire alarm sounds
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(stft), sr=sr, n_mfcc=13)

    # Calculate mean of MFCCs to summarize the audio characteristics
    mfccs_mean = np.mean(mfccs, axis=1)

    # Here, a threshold-based approach could be used to detect specific characteristics of a fire alarm sound
    # Example: Check for high-frequency patterns in MFCCs or other characteristics
    if np.any(mfccs_mean > 15):  # This is an arbitrary threshold, adjust based on real data
        print("Fire alarm sound detected!")
        return True

    print("No fire alarm detected.")
    return False

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("0.0.0.0", 5000, app)


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.227:5000
Press CTRL+C to quit
192.168.2.227 - - [29/Mar/2025 15:07:21] "POST /upload_image HTTP/1.1" 200 -
192.168.2.227 - - [29/Mar/2025 15:07:32] "POST /upload_image HTTP/1.1" 200 -
192.168.2.227 - - [29/Mar/2025 15:07:36] "POST /upload_image HTTP/1.1" 200 -
192.168.2.227 - - [29/Mar/2025 15:07:40] "POST /upload_image HTTP/1.1" 200 -
192.168.2.227 - - [29/Mar/2025 15:09:33] "POST /upload_audio HTTP/1.1" 200 -


Fire alarm sound detected!


192.168.2.227 - - [29/Mar/2025 15:09:50] "POST /upload_audio HTTP/1.1" 200 -


Fire alarm sound detected!


192.168.2.227 - - [29/Mar/2025 15:09:58] "POST /upload_audio HTTP/1.1" 200 -


Fire alarm sound detected!
